In [1]:
from ddm.processing import ddm
from dask import compute, delayed
from dask.diagnostics import ProgressBar
import nd2
import matplotlib.pyplot as plt

In [2]:
def load_data(file: str = "../data/A1_s3001.nd2", num_frames: int = 5000):
    f = nd2.ND2File(file)
    data = f.to_xarray(delayed=True)
    data = data.chunk({"T": 2})
    data = data[:num_frames,:,:]
    return data.astype("float64")

In [3]:
data = load_data()
# data

In [4]:
# %%time
# gTau = ddm(data, tau=2)
# out = gTau.compute()
# plt.plot(gTau)

In [5]:
# gTau.compute()

In [7]:
for tau in range(1,3):
    %%time gTau = compute(ddm(data, tau=tau))
    print(gTau[0][10])

44268852982.69997
56037799206.68083
CPU times: total: 2min 18s
Wall time: 1min 51s
